<a href="https://colab.research.google.com/github/anileshwar12/DiseasePrediction/blob/main/disease_symptoms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow pandas scikit-learn


In [2]:
from google.colab import files
uploaded = files.upload()


Saving DiseaseAndSymptoms.csv to DiseaseAndSymptoms.csv


In [3]:
import pandas as pd

df = pd.read_csv(next(iter(uploaded)))
df.fillna("", inplace=True)

symptom_cols = [col for col in df.columns if col.startswith("Symptom_")]

df["Symptoms"] = df[symptom_cols].values.tolist()
df["Symptoms"] = df["Symptoms"].apply(lambda x: [s.strip() for s in x if s.strip() != ""])

all_symptoms = sorted(set(symptom for row in df["Symptoms"] for symptom in row))
symptom_to_index = {symptom: i for i, symptom in enumerate(all_symptoms)}
input_dim = len(all_symptoms)


In [5]:
import tensorflow as tf
print(tf.__version__)


2.18.0


In [6]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

def encode_symptoms(symptoms):
    vec = [0] * input_dim
    for s in symptoms:
        if s in symptom_to_index:
            vec[symptom_to_index[s]] = 1
    return vec

df["input_vector"] = df["Symptoms"].apply(encode_symptoms)
X = np.array(df["input_vector"].tolist())

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df["Disease"])
y = to_categorical(y)
num_classes = y.shape[1]


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
    Dense(128, activation='relu', input_shape=(input_dim,)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=50, batch_size=16, validation_split=0.2)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.5162 - loss: 2.6722 - val_accuracy: 1.0000 - val_loss: 0.0315
Epoch 2/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9974 - loss: 0.0719 - val_accuracy: 1.0000 - val_loss: 0.0027
Epoch 3/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9993 - loss: 0.0190 - val_accuracy: 1.0000 - val_loss: 6.6285e-04
Epoch 4/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9997 - loss: 0.0091 - val_accuracy: 1.0000 - val_loss: 2.6886e-04
Epoch 5/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9998 - loss: 0.0056 - val_accuracy: 1.0000 - val_loss: 1.4158e-04
Epoch 6/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.0000 - loss: 0.0041 - val_accuracy: 1.0000 - val_loss: 7.0516e-05
Epoch 7/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.0000 - loss: 0.0025 - val_accuracy: 1.0000 - val_loss: 4.3213e-05
Epoch 8/50
246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 1.0000 - loss: 0.002

In [9]:
model.save('disease_model.keras')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("disease_model.tflite", "wb") as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmpuxgvm6k2'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 131), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 41), dtype=tf.float32, name=None)
Captures:
  134056758296208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134056753447568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134056753450448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134056753449296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134056753448720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134056753451024: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [10]:
import json

with open("symptom_index.json", "w") as f:
    json.dump(symptom_to_index, f)

with open("label_classes.json", "w") as f:
    json.dump(list(label_encoder.classes_), f)


In [11]:
from google.colab import files

files.download("disease_model.tflite")
files.download("symptom_index.json")
files.download("label_classes.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
interpreter = tf.lite.Interpreter(model_path="disease_model.tflite")
interpreter.allocate_tensors()


In [13]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input shape:", input_details[0]['shape'])
print("Input dtype:", input_details[0]['dtype'])


Input shape: [  1 131]
Input dtype: <class 'numpy.float32'>


In [16]:
test_input = np.zeros((1, 131), dtype=np.float32)
test_input[0, 121] = 1
test_input[0, 14] = 1
test_input[0, 109] = 1
test_input[0, 18] = 1


In [17]:
interpreter.set_tensor(input_details[0]['index'], test_input)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])


In [19]:
with open("label_classes.json", "r") as f:
    labels = json.load(f)

top_indices = output[0].argsort()[-4:][::-1]
for i in top_indices:
    print(f"{labels[i]}: {output[0][i]*100:.2f}%")


Heart attack: 100.00%
Chronic cholestasis: 0.00%
Tuberculosis: 0.00%
Paralysis (brain hemorrhage): 0.00%
